[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/i2i/transduction_bodies/observation_prediction_bodies.ipynb)

# Transduction from observation to prediction for bodies

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [5]:
# Preparation

# install tensor2tensor
! pip install -q -U tensor2tensor

# show versions
! pip show tensorflow
! pip show tensor2tensor

# get the needed resources
! curl -O http://josd.github.io/eye/i2i/transduction_bodies/observation_prediction_bodies.sh
! curl -O http://josd.github.io/eye/i2i/transduction_bodies/observation_prediction_bodies.py
! curl -O http://josd.github.io/eye/i2i/transduction_bodies/__init__.py
! curl -O http://josd.github.io/eye/i2i/transduction_bodies/test_bodies.observation
! chmod +x observation_prediction_bodies.sh

# clear data and model
% rm -fr /tmp/t2t_data/observation_prediction_bodies/
% rm -fr /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/

# start tensorboard
! curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip
get_ipython().system_raw('tensorboard --logdir /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Name: tensorflow
Version: 1.9.0rc2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: wheel, gast, termcolor, tensorboard, absl-py, astor, protobuf, setuptools, grpcio, six, numpy
Required-by: 
Name: tensor2tensor
Version: 1.6.6
Summary: Tensor2Tensor
Home-page: http://github.com/tensorflow/tensor2tensor
Author: Google Inc.
Author-email: no-reply@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: sympy, oauth2client, future, numpy, gevent, six, google-api-python-client, flask, scipy, requests, tqdm, h5py, gym, gunicorn, bz2file
Required-by: 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1311  100  1311    0     0   1311      0  0:00:0

In [6]:
# See the observation_prediction_bodies problem

! pygmentize -g observation_prediction_bodies.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionBodies(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for bodies."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 7,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 3,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(100000):
      # wind turbine size factor
      size_factor = 

In [7]:
# See the observation_prediction_bodies script

! pygmentize -g observation_prediction_bodies.sh

#!/bin/bash
PROBLEM=observation_prediction_bodies
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=/tmp/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=/tmp/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# train with Adam for 3600 steps
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --eval_throttle_seconds=30 \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=3600

# train with SGD for 3600 steps
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --eval_throttle_seconds=30 \
  --hparams="optimizer=SGD" \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=7200

# decode
t2t-decoder \
  --data_dir=$

In [8]:
# Run the observation_prediction_bodies script

! ./observation_prediction_bodies.sh

INFO:tensorflow:Importing user module content from path /
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9e02cbeb00>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow

INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-13-14:54:05
INFO:tensorflow:Graph was finalized.
2018-07-13 14:54:05.938355: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-07-13 14:54:05.938459: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-07-13 14:54:05.938492: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 
2018-07-13 14:54:05.938520: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N 
2018-07-13 14:54:05.938685: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, p

INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-13-14:55:04
INFO:tensorflow:Graph was finalized.
2018-07-13 14:55:04.596486: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-07-13 14:55:04.596576: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-07-13 14:55:04.596609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 
2018-07-13 14:55:04.596636: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N 
2018-07-13 14:55:04.596877: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, p

INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-13-14:56:01
INFO:tensorflow:Graph was finalized.
2018-07-13 14:56:02.262169: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-07-13 14:56:02.262261: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-07-13 14:56:02.262294: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 
2018-07-13 14:56:02.262322: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N 
2018-07-13 14:56:02.262475: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, p

INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-13-14:56:59
INFO:tensorflow:Graph was finalized.
2018-07-13 14:56:59.942613: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-07-13 14:56:59.942730: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-07-13 14:56:59.942764: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 
2018-07-13 14:56:59.942797: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N 
2018-07-13 14:56:59.942951: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring paramet

...

INFO:tensorflow:global_step/sec: 6.96671
INFO:tensorflow:loss = 0.012695564, step = 6738 (14.354 sec)
INFO:tensorflow:Saving checkpoints for 6778 into /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.021443712.
INFO:tensorflow:Reading data files from /tmp/t2t_data/observation_prediction_bodies/observation_prediction_bodies-dev*
INFO:tensorflow:partition: 0 num_data_files: 3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' 

INFO:tensorflow:global_step/sec: 7.03041
INFO:tensorflow:loss = 0.023104483, step = 6878 (14.225 sec)
INFO:tensorflow:Saving checkpoints for 6918 into /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.026387712.
INFO:tensorflow:Reading data files from /tmp/t2t_data/observation_prediction_bodies/observation_prediction_bodies-dev*
INFO:tensorflow:partition: 0 num_data_files: 3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' 

INFO:tensorflow:Saving checkpoints for 7061 into /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.024878722.
INFO:tensorflow:Reading data files from /tmp/t2t_data/observation_prediction_bodies/observation_prediction_bodies-dev*
INFO:tensorflow:partition: 0 num_data_files: 3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transf

INFO:tensorflow:Saving checkpoints for 7200 into /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.012289447.
INFO:tensorflow:Reading data files from /tmp/t2t_data/observation_prediction_bodies/observation_prediction_bodies-dev*
INFO:tensorflow:partition: 0 num_data_files: 3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transf

INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-07-13-15:45:55
INFO:tensorflow:Saving dict for global step 7200: global_step = 7200, loss = 0.016536655, metrics-observation_prediction_bodies/targets/accuracy = 0.9977381, metrics-observation_prediction_bodies/targets/accuracy_per_sequence = 0.98190475, metrics-observation_prediction_bodies/targets/accuracy_top5 = 1.0, metrics-observation_prediction_bodies/targets/approx_bleu_score = 0.9948225, metrics-observation_prediction_bodies/targets/neg_log_perplexity = -0.0896281, metrics-observation_prediction_bodies/targets/rouge_2_fscore = 0.99482995, metrics-observation_prediction_bodies/targets/rouge_L_fscore = 0.99773806
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7200: /tmp/t2t_train/observation_prediction_bodies/transfor

In [9]:
# See the transductions
# For each observation the top 2 predictions are shown with their respective log probability

! pygmentize -g test_bodies.observation
print("->-")
! pygmentize -g test_bodies.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.39	A_PERSON has BMI class O	-3.15
A_TURBINE producing 9533 kW	-0.52	A_TURBINE producing 5000 kW	-2.75


In [10]:
import os

import tensorflow as tf

from tensor2tensor import problems
from tensor2tensor.bin import t2t_decoder  # To register the hparams set
from tensor2tensor.utils import registry
from tensor2tensor.utils import trainer_lib
from tensor2tensor.visualization import attention
from tensor2tensor.visualization import visualization

In [11]:
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

In [12]:
# MODEL
CHECKPOINT = os.path.expanduser('/tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small')

In [13]:
# HPARAMS
problem_name = 'observation_prediction_bodies'
data_dir = os.path.expanduser('/tmp/t2t_data/observation_prediction_bodies')
model_name = "transformer"
hparams_set = "transformer_small"

In [14]:
import observation_prediction_bodies

visualizer = visualization.AttentionVisualizer(hparams_set, model_name, data_dir, problem_name, beam_size=1)

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_873_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_873_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_873_256.top
INFO:tensorflow:Greedy Decoding


In [15]:
tf.Variable(0, dtype=tf.int64, trainable=False, name='global_step')

sess = tf.train.MonitoredTrainingSession(
    checkpoint_dir=CHECKPOINT,
    save_summaries_secs=0,
)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt-7200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 7200 into /tmp/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.


In [16]:
input_sentence = "A_TURBINE with size factor 4 and subjected to windspeed 62 km/h"
output_string, inp_text, out_text, att_mats = visualizer.get_vis_data_from_string(sess, input_sentence)
print(output_string)

A_TURBINE producing 9533 kW<EOS>


## Interpreting the Visualizations
- The layers drop down allow you to view the different Transformer layers, 0-indexed of course.
  - Tip: The first layer, last layer and 2nd to last layer are usually the most interpretable.
- The attention dropdown allows you to select different pairs of encoder-decoder attentions:
  - All: Shows all types of attentions together. NOTE: There is no relation between heads of the same color - between the decoder self attention and decoder-encoder attention since they do not share parameters.
  - Input - Input: Shows only the encoder self-attention.
  - Input - Output: Shows the decoder’s attention on the encoder. NOTE: Every decoder layer attends to the final layer of encoder so the visualization will show the attention on the final encoder layer regardless of what layer is selected in the drop down.
  - Output - Output: Shows only the decoder self-attention. NOTE: The visualization might be slightly misleading in the first layer since the text shown is the target of the decoder, the input to the decoder at layer 0 is this text with a GO symbol prepreded.
- The colored squares represent the different attention heads.
  - You can hide or show a given head by clicking on it’s color.
  - Double clicking a color will hide all other colors, double clicking on a color when it’s the only head showing will show all the heads again.
- You can hover over a word to see the individual attention weights for just that position.
  - Hovering over the words on the left will show what that position attended to.
  - Hovering over the words on the right will show what positions attended to it.

In [17]:
call_html()
attention.show(inp_text, out_text, *att_mats)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>